In [108]:
import pandas as pd

fileGenre = open('../data/u.genre', 'r')
fileItem = open('../data/u.item', 'r')
fileUser = open('../data/u.user', 'r')

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        


        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gendre'] = rec[2]
            item['function'] = rec[4]
        else:
            return

        data.append(item)
    filed.close()
    return data

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)


df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)

In [109]:

fileData = open('../data/u.data', 'r')
def getDataframeFromFileWithSpaces(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

relationship = getDataframeFromFileWithSpaces(fileData)
df_relations = pd.DataFrame(relationship)

In [110]:
merged_df = pd.merge(df_movies, df_relations[['userId', 'movieId', 'rating','timestamp']], on='movieId', how='inner')

In [128]:
merged_df[(merged_df['movieId'] == '1682') & (merged_df['userId'] == '916')]

,movieId,name,release_date,url,genre,userId,rating,timestamp
99999,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,http://us.imdb.com/M/title-exact?Schrei%20aus%...,[Drama],916,3,880845755


<hr>
<p style='color:red;'> Note: The input data must be modeled in the following way :</p>
<code>
{"userId": "365",
  "movie": 
    { "movieId": "7526",
        "title": "Lord of the Rings: The Fellowship of the Ring, The (2001)",
        "genres": ["Adventure", "Fantasy"] 
    },
    "rating": "5",
    "timestamp": "45554125" 
}
</code>
<p style='color:red;'>  Ouput:</p>
<hr>

In [158]:
movie_list = []
for x in merged_df.itertuples():
    items = {"userId": x.userId,
        "movie": 
        { "movieId": x.movieId,
            "title": x.name,
            "genres": x.genre
        },
        "rating": x.rating,
        "timestamp": x.timestamp
    }
    movie_list.append(items)
movie_list[0:5]

[{'userId': '308',
  'movie': {'movieId': '1',
   'title': 'Toy Story (1995)',
   'genres': ['Animation', "Children's", 'Comedy']},
  'rating': '4',
  'timestamp': '887736532'},
 {'userId': '287',
  'movie': {'movieId': '1',
   'title': 'Toy Story (1995)',
   'genres': ['Animation', "Children's", 'Comedy']},
  'rating': '5',
  'timestamp': '875334088'},
 {'userId': '148',
  'movie': {'movieId': '1',
   'title': 'Toy Story (1995)',
   'genres': ['Animation', "Children's", 'Comedy']},
  'rating': '4',
  'timestamp': '877019411'},
 {'userId': '280',
  'movie': {'movieId': '1',
   'title': 'Toy Story (1995)',
   'genres': ['Animation', "Children's", 'Comedy']},
  'rating': '4',
  'timestamp': '891700426'},
 {'userId': '66',
  'movie': {'movieId': '1',
   'title': 'Toy Story (1995)',
   'genres': ['Animation', "Children's", 'Comedy']},
  'rating': '3',
  'timestamp': '883601324'}]

In [138]:
merged_df[(merged_df['movieId'] == '1') & (merged_df['userId'] == '280')]

,movieId,name,release_date,url,genre,userId,rating,timestamp
3,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"[Animation, Children's, Comedy]",280,4,891700426
